In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize': [10, 10]}, font_scale=1.3)
pd.set_option('mode.chained_assignment', None)

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(r"F:\Data Science\Course\football project2\international_matches.csv")
df.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,tournament,city,country,neutral_location,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,1,Friendly,Maceió,Brazil,False,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,0,FIFA World Cup qualification,Quito,Ecuador,False,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,0,Friendly,Conakry,Guinea,False,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cols = df.columns[-8:]
cols

Index(['home_team_goalkeeper_score', 'away_team_goalkeeper_score',
       'home_team_mean_defense_score', 'home_team_mean_offense_score',
       'home_team_mean_midfield_score', 'away_team_mean_defense_score',
       'away_team_mean_offense_score', 'away_team_mean_midfield_score'],
      dtype='object')

In [4]:
df.drop(cols, axis=1, inplace=True)

In [5]:
result_dict = {'Win':2,'Lose':0,'Draw':1} 

df["Home_Team_Result"] = df["home_team_result"].map(result_dict)

In [6]:
def away_team_result(result):
    if result == "Win":
        return "Lose"
    elif result == "Lose":
        return "Win"
    else:
        return "Draw"
    
df["away_team_result"] = df["home_team_result"].apply(away_team_result)

In [7]:
def Team(team):
    team_name = df["home_team"].unique()
    team_number = range(211)
    teams = dict(zip(team_name,team_number))
    team = teams[team]
    return team

In [8]:
df["home_team_number"] = df["home_team"].apply(Team)
df["away_team_number"] = df["away_team"].apply(Team)

In [9]:
def matches_statistics(teams, statisitcs):
    statistics_results = []
    for new_team in df[f"{teams}"]:
        for team,rank_or_points in zip(statisitcs.index,statisitcs.values):
            if new_team == team:
                new_team = rank_or_points
                statistics_results.append(int(new_team))
            else:
                continue
    return statistics_results

In [10]:
def Rank(df, teams, fifa_ranks):
    mean_ranks = df.groupby(by=f"{teams}")[f"{fifa_ranks}"].mean()
    mean_ranks = matches_statistics(teams, mean_ranks)
    return mean_ranks

In [11]:
df["mean_home_team_rank"] = Rank(df,'home_team',"home_team_fifa_rank")
df["mean_away_team_rank"] = Rank(df,'away_team',"away_team_fifa_rank")

In [12]:
def Points(df, teams, total_points):
    max_points = df.groupby(by=f"{teams}")[f"{total_points}"].max()
    max_points = matches_statistics(teams, max_points)
    return max_points

In [13]:
df['home_team_max_points'] = Points(df,'home_team','home_team_total_fifa_points')
df['away_team_max_points'] = Points(df,'away_team','away_team_total_fifa_points')

In [14]:
def matches_Dict(teams):
    Dict = {}
    for team1 in df[teams]:
        for team2,matches in zip(df[teams].value_counts().index.tolist(),df[teams].value_counts()):
            if team1 == team2:
                team1 = matches
                Dict[team2] = []
                Dict[team2].append(team1)
                Dict[team2] = sum(Dict[team2])
    return Dict

In [15]:
def number_of_matches(teams, Dict):
    team_matches = []
    for team1 in df[teams]:
        for team2, matches in zip(Dict.keys(),Dict.values()):
            if team1 == team2:
                team1 = matches
                team_matches.append(team1)

    return team_matches

In [16]:
def number_of_all_matches(df):
    all_matches_home_and_away = {}
    for team1 in df["home_team"].unique():
        for team2 in df["away_team"].unique():
            match1 = df[df["home_team"] == f"{team1}"]["home_matches"].unique()[0]
            match2 = df[df["away_team"] == f"{team2}"]["away_matches"].unique()[0]
            if team1 == team2:
                matches = match1+match2
                all_matches_home_and_away[team1] = []
                all_matches_home_and_away[team1].append(matches)
                all_matches_home_and_away[team1] = sum(all_matches_home_and_away[team1])

    return all_matches_home_and_away

In [17]:
def result(df, teams, Dict):
    if Dict == matches_Dict:
        Dict = Dict(teams)
    elif Dict == number_of_all_matches:
        Dict = Dict(df)
    team_matches = number_of_matches(teams, Dict)
    return team_matches

In [18]:
df["home_matches"] = result(df, "home_team", matches_Dict)

In [19]:
df["home_matches"]

0        126
1        233
2        146
3        103
4        121
        ... 
23916    109
23917     98
23918    168
23919    280
23920    249
Name: home_matches, Length: 23921, dtype: int64

In [20]:
df["away_matches"] = result(df, "away_team", matches_Dict)

In [21]:
df["home_team_matches"] = result(df, "home_team", number_of_all_matches)

In [22]:
df["away_team_matches"] = result(df, "away_team", number_of_all_matches)

In [23]:
df

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,tournament,city,country,neutral_location,shoot_out,home_team_result,Home_Team_Result,away_team_result,home_team_number,away_team_number,mean_home_team_rank,mean_away_team_rank,home_team_max_points,away_team_max_points,home_matches,away_matches,home_team_matches,away_team_matches
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,No,Win,2,Lose,0,12,68,21,1374,1645,126,194,285,333
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,1,Friendly,Maceió,Brazil,False,No,Draw,1,Draw,1,32,3,14,2160,2041,233,201,433,517
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,0,FIFA World Cup qualification,Quito,Ecuador,False,No,Win,2,Lose,2,29,41,71,1458,1517,146,169,335,305
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,0,Friendly,Conakry,Guinea,False,No,Win,2,Lose,3,67,70,103,1354,1373,103,84,260,150
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,No,Lose,0,Win,4,13,34,5,1484,1766,121,177,337,367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23916,2022-06-14,Moldova,Andorra,Europe,Europe,180,153,932,1040,2,1,UEFA Nations League,Chișinău,Moldova,False,No,Win,2,Lose,122,177,122,166,985,1121,109,92,251,189
23917,2022-06-14,Liechtenstein,Latvia,Europe,Europe,192,135,895,1105,0,2,UEFA Nations League,Vaduz,Liechtenstein,False,No,Lose,0,Win,57,105,155,90,944,1122,98,152,213,292
23918,2022-06-14,Chile,Ghana,South America,Africa,28,60,1526,1387,0,0,Kirin Cup,Suita,Japan,True,Yes,Lose,0,Win,98,71,28,39,1579,1439,168,198,359,349
23919,2022-06-14,Japan,Tunisia,Asia,Africa,23,35,1553,1499,0,3,Kirin Cup,Suita,Japan,False,No,Lose,0,Win,39,35,34,34,1553,1526,280,164,425,362


In [206]:
def how_many_wins_or_loses(df,win_or_loss="Win",team="home_team"):
    if win_or_loss=="Win": 
        if team=="home_team":
            return df[df["home_team_result"] == "Win"].count()[0]
        elif team=="away_team": 
            return df[df["away_team_result"] == "Win"].count()[0]
    elif win_or_loss=="Loss": 
        if team=="home_team":
            return df[df["home_team_result"] == "Loss"].count()[0]
        elif team=="away_team": 
            return df[df["away_team_result"] == "Loss"].count()[0]

In [286]:
def unique_teams():
    how_many_wins_or_loses(df,"Win","home_team")
    group_unique_teams = df.groupby(by="home_team").apply(how_many_wins_or_loses, args=(df,"Win","home_team"))
    return group_unique_teams

In [392]:
def how_many_wins_or_loses(df,win_or_loss,team="home_team"):
    if win_or_loss=="Win": 
        if team=="home_team":
            result = df[df["home_team_result"] == "Win"].count()[0]
        elif team=="away_team": 
            result = df[df["away_team_result"] == "Win"].count()[0]
    elif win_or_loss=="Loss": 
        if team=="home_team":
            result = df[df["home_team_result"] == "Loss"].count()[0]
        elif team=="away_team": 
            result = df[df["away_team_result"] == "Loss"].count()[0]
    group_unique_teams = df.groupby(by=team).apply(result)
    return group_unique_teams

In [393]:
t = how_many_wins_or_loses(df,"Win","home_team")

TypeError: 'numpy.int64' object is not callable

In [387]:
def unique_teams(df,Win,home_teams):
    group_unique_teams = df.groupby(by=teams).apply(how_many_wins_or_loses, (df,win,teams))
    return group_unique_teams

In [388]:
def this_team(df,win_or_loss="Win",team="home_team"):
    #fun = how_many_wins_or_loses
    z = unique_teams.apply(how_many_wins_or_loses, (df,win,teams))
    return z

In [389]:
this_team(df=df)

AttributeError: 'function' object has no attribute 'apply'

In [122]:
unique_teams(df)

TypeError: how_many_wins_or_loses() missing 2 required positional arguments: 'win_or_loss' and 'team'

In [117]:
def x(n):
    if n == "Team":
        return 1
        
v = x("Team")
v

1

In [48]:
x

home_team
Afghanistan       16
Albania           52
Algeria           91
American Samoa     3
Andorra           10
                  ..
Vietnam           57
Wales             53
Yemen             27
Zambia            94
Zimbabwe          74
Length: 211, dtype: int64

In [40]:
home_team_win = []
for team1 in df["home_team"]:
    for team2, win in zip(x.index, x.values):
        if team1 == team2:
            team1 = win
            home_team_win.append(team1)
            
df["home_team_win"] = home_team_win

In [41]:
df["home_team_win"]

0         50
1        181
2         83
3         61
4         64
        ... 
23916     22
23917      9
23918     99
23919    154
23920    140
Name: home_team_win, Length: 23921, dtype: int64

In [42]:
def home_team_winning_percentage(df):
    result = df["home_team_win"]/df["home_matches"]
    return result.round(2)

df["home_team_winning_percentage"] = df[["home_team_win", "home_matches"]].apply(home_team_winning_percentage, axis=1)

In [44]:
df["home_team_winning_percentage"]

0        0.40
1        0.78
2        0.57
3        0.59
4        0.53
         ... 
23916    0.20
23917    0.09
23918    0.59
23919    0.55
23920    0.56
Name: home_team_winning_percentage, Length: 23921, dtype: float64